# 3D MAPS WITH PYDECK
For more information on pydeck, visit the python package index or visit the documentation at https://pydeck.gl.

# DOCUMENTATION

In [1]:
from IPython.display import IFrame
documentation = IFrame(src='https://pydeck.gl', width=1000, height=500)
display(documentation)

# NOTES

In [2]:
# Mapbox requires an API key
# command + scroll to rotate map (different on Windows and other combinations work also - shift + scroll)

# PATTERN TO CREATE MAPS
# data
# view (location, zoom level, etc.)
# layer
# render map
# display and save map (to_html(), show())

# IMPORTS

In [3]:
import pydeck as pdk
import pandas as pd
from vega_datasets import data as vds
import ipywidgets
# use palettable for colormap colors
from palettable.cartocolors.sequential import BrwnYl_3
import json

# enter your api key here
api_token = ("pk.eyJ1IjoiYW5zaHVsdG1yIiwiYSI6ImNrc2VxeGEzOTEzOHUycHAzbjhiYW9lNTYifQ.42Mrl0G1PDh7nrjV4M6Gkw")

# API key required for mapbox basemap tiles
#MAPBOX_API_KEY = api_token

# To enable pydeck for Jupyter Notebook:
# jupyter nbextension install --sys-prefix --symlink --overwrite --py pydeck
# jupyter nbextension enable --sys-prefix --py pydeck

# COLUMNLAYER EXAMPLE - POPULATION BY COUNTY

In [4]:
gps_data = "clean_data.csv"
gps_df = pd.read_csv(gps_data)

In [5]:
# data
# in this example, the population was scaled down
# to change visualization, try changing the elevation_scale and radius arguments

gps_df['result'] = gps_df['result']/1_000
print(gps_df.columns)

# view (location, zoom level, etc.)
view = pdk.ViewState(latitude=39.155726, longitude=-98.030561, pitch=50, zoom=3)

# layer
column_layer = pdk.Layer('ColumnLayer',
                         data=gps_df,
                         get_position=['long', 'lat'],
                         get_elevation='result',
                         elevation_scale=100,
                         radius=5000,
                         get_fill_color=[255, 165, 0, 80],
                         pickable=True,
                         auto_highlight=True)

# render map
# with no map_style, map goes to default
column_layer_map = pdk.Deck(layers=column_layer, 
                            initial_view_state=view)
                            mapbox_key=api_token)

# display and save map (to_html(), show())
column_layer_map.show()

Index(['@iot.id', 'phenomenonDate', 'result', 'id', 'lat', 'long', 'weekday'], dtype='object')


DeckGLWidget(carto_key=None, custom_libraries=[], google_maps_key=None, json_input='{"initialViewState": {"lat…

In [6]:
column_layer_map.show()

DeckGLWidget(carto_key=None, custom_libraries=[], google_maps_key=None, json_input='{"initialViewState": {"lat…

# HEATMAPLAYER

In [ ]:
# data
data = 'https://raw.githubusercontent.com/groundhogday321/dataframe-datasets/master/us_population_by_city_2018.csv'
city_population = pd.read_csv(data)
print(city_population.head())

# view (location, zoom level, etc.)
view = pdk.ViewState(latitude=39.155726, longitude=-98.030561, pitch=50, zoom=3)

# layer
heatmap_layer = pdk.Layer('HeatmapLayer',
                          data=city_population,
                          opacity=0.9,
                          get_position=['longitude', 'latitude'],
                          color_range=BrwnYl_3.colors,
                          threshold=0.2,
                          get_weight='Population_2018',
                          pickable=True)

# render map
heatmap_layer_map = pdk.Deck(layers=heatmap_layer,
                             initial_view_state=view,
                             mapbox_key=MAPBOX_API_KEY)

# display and save map (to_html(), show())
heatmap_layer_map.show()

# ANIMATED
### ADD WIDGET FOR INTERACTIVE MAP WITH ANIMATION

In [ ]:
# data
# in this example, the population was scaled down
data = 'https://raw.githubusercontent.com/groundhogday321/dataframe-datasets/master/gapminder_centroids.csv'
gapminder = pd.read_csv(data)
gapminder['scaled_population'] = gapminder['pop']/50_000

# view (location, zoom level, etc.)
view = pdk.ViewState(latitude=39, longitude=35, pitch=50, zoom=1)

# layer
gapminder_layer = pdk.Layer('ColumnLayer',
                             data=gapminder,
                             get_position=['longitude', 'latitude'],
                             get_elevation='scaled_population',
                             elevation_scale=100,
                             radius=200_000,
                             get_fill_color=[255, 165, 0, 80],
                             pickable=True,
                             auto_highlight=True)

# render map
# with no map_style, map goes to default
gapminder_layer_map = pdk.Deck(layers=gapminder_layer, 
                               initial_view_state=view,
                               mapbox_key=MAPBOX_API_KEY)

# display and save map (to_html(), show())
# display(gapminder_layer_map.show())
# display at botton

# widgets
slider = ipywidgets.IntSlider(value=1955, min=1955, max=2005, step=5)
play = ipywidgets.Play(value=1955, min=1955, max=2005, step=5, description='Press play', interval=1_000)
ipywidgets.jslink((play, 'value'), (slider, 'value'))
layout = ipywidgets.HBox([slider, play])

# function
def update_plot(year):
    gapminder_layer.data = gapminder[gapminder['year'] == year]
    return gapminder_layer_map.update()

# interaction between widget and function
interact = ipywidgets.interactive_output(update_plot, {'year': slider});
display(layout, interact)

# display and save map (to_html(), show())
gapminder_layer_map.show()